In [44]:
from threading import Timer

text = 1

def credito_predict():
    
    #importing packages
    import numpy as np
    import pandas as pd
    from sklearn.model_selection import train_test_split
    from sklearn.model_selection import cross_val_score
    from sklearn.metrics import accuracy_score
    from sklearn.linear_model import LogisticRegression
    from sklearn.linear_model import LinearRegression

    import firebase_admin                      
    from firebase_admin import credentials
    from firebase_admin import firestore

    global text
    text+=1
    print(f"{text}")

    #----------------------------------------------------------------------------------------------------#
    # reading csv file
    data = pd.read_csv("/content/loan_data2.csv")  
    df=data
    
    ## Filling up missing values    
    df['Gender'] = df['Gender'].fillna('Male')
    df['Dependents'] = df['Dependents'].fillna('0')
    df['Self_Employed']=df['Self_Employed'].fillna('No')
    df['Married'] = df['Married'].fillna('Yes')
    
    mean=df[df['LoanAmount']<=400]['LoanAmount'].mean()
    df['LoanAmount'].fillna(mean,inplace=True)
    df['Loan_Amount_Term'].fillna(df['Loan_Amount_Term'].value_counts().idxmax(), inplace=True)
    df['Credit_History'].fillna(df['Credit_History'].value_counts().idxmax(), inplace=True)
    
    # Converting categorical values to numerical values so that we can use sklearn
    ds=df.replace(to_replace=["Yes","No"],value=['1','0'])
    ds=ds.replace(to_replace=["Y","N"],value=['1','0'])
    ds=ds.replace(to_replace=["Graduate","Not Graduate"],value=['1','0'])
    ds=ds.replace(to_replace=["Rural","Semiurban","Urban"],value=['0','1','2'])
    ds=ds.replace(to_replace=["Male","Female"],value=['1','0'])
    #ds.head(10)
   #----------------------------------------------------------------------------------------------------# 
    #Scaling values
    from sklearn.preprocessing import StandardScaler
    
    def scale(x_train,x_test):
        std = StandardScaler()
        std.fit(x_train)
        xtr = std.transform(x_train)
        xts = std.transform(x_test)
        return xtr,xts,std;
    
    x=ds[['ApplicantIncome','CoapplicantIncome','LoanAmount','Credit_History','Property_Area']]
    y=ds['Loan_Status']
    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)
    xtr,xts,std=scale(x_train,x_test)
    #----------------------------------------------------------------------------------------------------#
    #Function for Logistic Regression Model
    def logisticmodel(xtr,xts,y_train):
        model = LogisticRegression()
        model.fit(xtr , y_train)
        predict = model.predict(xts)
        return predict;
    
    predict=logisticmodel(xtr,xts,y_train)    #calling logistic regression function
    #----------------------------------------------------------------------------------------------------#
    
    ##User Input
    #print("Enter the following details :")
    ap_in=0                      #int(input("Enter Applicant income :"))
    coapp_in=0                   #float(input("Enter Coapplicant income :"))
    loan=0                       #float(input("Enter Desired loan amount :"))
    credh=0                      #float(input("Enter credit history [1.0/0.0] :"))
    prop_area=''                 #input("Enter property area [0/1/2] :")
    
    #######################################################################################################
    ## User input from firebase    # Use a service account
   
    if not firebase_admin._apps:                                         #print(firebase_admin._apps)
        cred = credentials.Certificate('/content/serviceAccountKey3.json') 
        default_app = firebase_admin.initialize_app(cred)
    db = firestore.client()
    
    
    doc_ref = db.collection('Users')
    docs = doc_ref.stream()    # getting all doc references of particular collection using .get() method
    
    uid=" "                 # selecting uid whose flag=1
    for j in docs:
        dic = j.to_dict()
        if(dic['flag']=='1'):
            uid=j.id
            break
    #print(uid)
    
    if(uid!=" "):          # checking if any doc has flag 1, if not then no prediction
        req_doc = doc_ref.document(uid).get()    # now getting the reference of uid with flag=1 
        req_doc = req_doc.to_dict()
        #print(req_doc)
        #detail = req_doc['Personal_Data']
        #print(doc,"\n",detail)

        for x in req_doc:
            if(x=='A_income'):
                ap_in=req_doc[x]

            elif(x=='Co_income'):
                coapp_in=req_doc[x]

            elif(x=='LoanAmount'):
                loan=req_doc[x]

            elif(x=='Credit_History'):
                credh=req_doc[x]

            elif(x=='Property_area'):    
                if(req_doc[x]=='Rural'):
                    prop_area=0
                elif(req_doc[x]=='Semiurban'):
                    prop_area=1
                else:
                    prop_area=2

        ########################################################################################################

        ## Dataframe to store the user input
        datta = [[ap_in,coapp_in,loan,credh,prop_area]]
        new_datta = pd.DataFrame(datta, columns = ['ApplicantIncome','CoapplicantIncome','LoanAmount','Credit_History','Property_Area'])  

        ## Scalling User input
        new_data = std.transform(new_datta)

        ## Predicting Loan Status and suitable loan amount using Logistic and Linear Regression model
        def linearreg(x_rtrain,y_rtrain,new_datta):
            model = LinearRegression()
            model.fit(x_rtrain, y_rtrain)
            r_sq = model.score(x_rtrain, y_rtrain)
            #print(r_sq)
            y_pred = model.predict(new_datta[['ApplicantIncome','CoapplicantIncome']])
            return y_pred
            #print('\nSuitable Loan Amount :', y_pred)
            #print("\n")

        predict=logisticmodel(xtr,new_data,y_train)
        # print(predict)

        doc_ref = db.collection('Users').document(uid)   # getting collection and document reference from firebase

        if(predict=='1'):
            #print(apr)
            xreg=ds[['ApplicantIncome','ApplicantIncome']]
            yreg=ds['LoanAmount']
            x_rtrain,x_rtest,y_rtrain,y_rtest=train_test_split(xreg,yreg,test_size=0.2,random_state=0)
            x_rtrain=x_rtrain.values
            y_rtrain=y_rtrain.values
            amount=linearreg(x_rtrain,y_rtrain,new_datta)

            doc_ref.update({                # updating values in firebase
                "flag":"0",
                "loan_status":"Approved",
                "Predicted_loan_amount":str(amount[0])
            })
        else:
            doc_ref.update({           # updating values in firebase
                "flag":"0",
                "loan_status":"Disapproved",
                "Predicted_loan_amount":"0"
            })
    Timer(5,credito_predict).start()
    
    
credito_predict()          # calling function

62
2
